In [9]:
import import_ipynb
from utils import request_influxdb
from datetime import datetime, timezone, timedelta
import pandas as pd
from io import StringIO
from pm25_to_aqi import equation1


In [10]:
# Load Tangaras
tangaras = pd.read_csv('../db/csv/tangara.csv')
tangaras.shape

(25, 13)

In [11]:
tangaras.head()

,id,mac,geohash,codigo,latitud,longitud,id_barrio,id_sector,id_areaexp,id_areapro,online,created,updated
0,1,D29ESP32DE02BBA,d29e6b4,TANGARA_2BBA,3.384476,-76.516342,NaN,NaN,NaN,NaN,0,2023-05-28T17:42:45.488386-05:00,2023-05-28T17:42:45.488386-05:00
1,2,D29ESP32DE0DD66,d29dbmw,TANGARA_DD66,3.326797,-76.620712,NaN,NaN,NaN,NaN,0,2023-05-28T17:42:45.488386-05:00,2023-05-28T17:42:45.488386-05:00
2,3,D29ESP32DE0E752,d29dbmw,TANGARA_E752,3.326797,-76.620712,NaN,NaN,NaN,NaN,0,2023-05-28T17:42:45.488386-05:00,2023-05-28T17:42:45.488386-05:00
3,4,D29ESP32DE1260A,d29dbmw,TANGARA_260A,3.326797,-76.620712,NaN,NaN,NaN,NaN,0,2023-05-28T17:42:45.488386-05:00,2023-05-28T17:42:45.488386-05:00
4,5,D29ESP32DE94B1A,d29dbmw,TANGARA_4B1A,3.326797,-76.620712,NaN,NaN,NaN,NaN,0,2023-05-28T17:42:45.488386-05:00,2023-05-28T17:42:45.488386-05:00


In [12]:
mac_addresses = tangaras['mac'].to_list()

In [13]:
def pm25_realtime(mac_addresses):
    # ISO 8601 Format, TZ='America/Bogota' -05:00, Last 5 minutes
    tz_bogota_co = timezone(offset=-timedelta(hours=5), name='America/Bogota')
    time_delta = timedelta(minutes=5)
    # Start DateTime
    start_datetime = (datetime.now(tz=tz_bogota_co) - time_delta).timestamp()
    start_datetime = int(start_datetime) * 1000
    # End DateTime
    end_datetime = datetime.now(tz=tz_bogota_co).timestamp()
    end_datetime = int(end_datetime) * 1000
    # Perdiod DateTime
    period_time = f"time >= {start_datetime}ms AND time <= {end_datetime}ms"
    # SQL Query
    sql_query = f"SELECT mean(\"pm25\") " \
                "FROM \"fixed_stations_01\" WHERE ("
    for mac in mac_addresses:
        sql_query += f"\"name\" = '{mac}' OR "
    sql_query = sql_query[:-4]
    sql_query += f") AND " \
                 f"{period_time} " \
                 f"GROUP BY time(1m) fill(none);"
    # InfluxDB API REST Request
    influxdb_request = request_influxdb(sql_query)
    # DataFrame last 5 minutes
    df_realtime = pd.read_csv(StringIO(influxdb_request.text), sep=",", low_memory=False)
    #print(df_realtime)
    # Remove/Add Columns
    df_realtime = df_realtime[['time', 'mean']]
    df_realtime.rename(columns={'time': 'DATETIME', 'mean': 'PM25'}, inplace=True)
    # Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
    df_realtime['DATETIME'] = df_realtime['DATETIME'].apply(lambda x: datetime.fromtimestamp(int(x) / 1000, tz=tz_bogota_co).isoformat())
    # AQI
    df_realtime['AQI'] = df_realtime['PM25'].apply(lambda x: equation1(x))
    #
    return df_realtime


In [14]:
# PM25 realtime, last 5 minures
df_realtime = pm25_realtime(mac_addresses)

pm25_mean = df_realtime['PM25'].mean()
print('pm25_mean:', round(pm25_mean, 2))
print('aqi_mean:', equation1(pm25_mean))

df_realtime.head()

pm25_mean: 12.3
aqi_mean: 52


,DATETIME,PM25,AQI
0,2023-06-03T22:37:00-05:00,13.307692,54
1,2023-06-03T22:38:00-05:00,13.035714,53
2,2023-06-03T22:39:00-05:00,12.607143,53
3,2023-06-03T22:40:00-05:00,12.607143,53
4,2023-06-03T22:41:00-05:00,12.750000,53


In [15]:
def pm25_last_1_hour(mac_addresses):
    # ISO 8601 Format, TZ='America/Bogota' -05:00, Last 1 hour
    tz_bogota_co = timezone(offset=-timedelta(hours=5), name='America/Bogota')
    time_delta = timedelta(hours=1)
    # Start DateTime
    start_datetime = (datetime.now(tz=tz_bogota_co) - time_delta).timestamp()
    start_datetime = int(start_datetime) * 1000
    # End DateTime
    end_datetime = datetime.now(tz=tz_bogota_co).timestamp()
    end_datetime = int(end_datetime) * 1000
    # Perdiod DateTime
    period_time = f"time >= {start_datetime}ms AND time <= {end_datetime}ms"
    # SQL Query
    sql_query = f"SELECT mean(\"pm25\") " \
                "FROM \"fixed_stations_01\" WHERE ("
    for mac in mac_addresses:
        sql_query += f"\"name\" = '{mac}' OR "
    sql_query = sql_query[:-4]
    sql_query += f") AND " \
                 f"{period_time} " \
                 f"GROUP BY time(1h) fill(none);"
    # InfluxDB API REST Request
    influxdb_request = request_influxdb(sql_query)
    # DataFrame last 1 hour
    df_last_1_hour = pd.read_csv(StringIO(influxdb_request.text), sep=",", low_memory=False)
    #print(df_last_1_hour)
    # Remove/Add Columns
    df_last_1_hour = df_last_1_hour[['time', 'mean']]
    df_last_1_hour.rename(columns={'time': 'DATETIME', 'mean': 'PM25'}, inplace=True)
    # Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
    df_last_1_hour['DATETIME'] = df_last_1_hour['DATETIME'].apply(lambda x: datetime.fromtimestamp(int(x) / 1000, tz=tz_bogota_co).isoformat())
    # AQI
    df_last_1_hour['AQI'] = df_last_1_hour['PM25'].apply(lambda x: equation1(x))
    #
    return df_last_1_hour


In [16]:
# PM25 Last Hour, last 1 hour
df_last_1_hour = pm25_last_1_hour(mac_addresses)

pm25_mean = df_last_1_hour['PM25'].mean()
print('pm25_mean:', round(pm25_mean, 2))
print('aqi_mean:', equation1(pm25_mean))

df_last_1_hour.head()

pm25_mean: 11.64
aqi_mean: 49


,DATETIME,PM25,AQI
0,2023-06-03T21:00:00-05:00,10.876768,46
1,2023-06-03T22:00:00-05:00,12.394604,52
